# <a href="https://girafe.ai/" target="_blank" rel="noopener noreferrer"><img src="https://raw.githubusercontent.com/girafe-ai/ml-mipt/7096a5df4cada5ee651be1e3215c2f7fb8a7e0bf/logo_margin.svg" alt="girafe-ai logo" width="150px" align="left"></a> [ml-mipt](https://github.com/girafe-ai/ml-mipt) basic course <a class="tocSkip">

# Seminar 01: kNN with scikit-learn <a class="tocSkip">

Today we will dig into k Nearest Neigbours algorithm for classification and regression.

We consider this algorithm implementation from widely known [`scikit-learn` library](https://scikit-learn.org/stable/index.html) and explore basic concepts of this library which we will extensively use during the whole course.

# `scikit-learn` introduction

## Origins

To cite [History page](https://scikit-learn.org/stable/about.html#history):
> This project was started in 2007 as a Google Summer of Code project by David Cournapeau. Later that year, Matthieu Brucher started work on this project as part of his thesis.

In 2010 various authors of INRIA took leadership of the project and made the **first public release, February the 1st 2010**.</br>
Since then, several releases have appeared following a ~ 3-month cycle, and a thriving international community has been leading the development.</br>
Long awaited **verison 1.0.0 was released at September, 24th of 2021.**

Now project's code is [available on github](https://github.com/scikit-learn/scikit-learn) (49k stars).

_Tip:_ If you don't know how some algorithm works in detail, you could easily go to source code and investigate what you need.

## Installation

This is regular PyPI package, so simple

In [ ]:
!pip install scikit-learn

goes well.</br>
As well as `conda` variant:

`conda install -c conda-forge scikit-learn`

## First glance

`sklearn` (shortened nickname for scikit-learn) implement most of classical and frequently used algorithms in Machine Learning.</br>
Also it provides [User Guide](https://scikit-learn.org/stable/user_guide.html) describing principles, mechanisms and references of every bunch of algorithms implemented.

As an entry point to main `sklearn`'s concepts **we recommend [getting started tutorial](https://scikit-learn.org/stable/getting_started.html)**.</br>
**Check it out:** it contains main concepts we will need for the whole course!!!</br>

[Further tutorials](https://scikit-learn.org/stable/tutorial/index.html) can also be handy to develop your skills.

In [ ]:
# regular machine learning libraries imports
import matplotlib.pyplot as plt
import numpy as np

# Synthetic dataset for classification

Before applying any model we need some data to operate upon.</br>
Let's generate toy task - only 2 features for better visualization.

To do that with `sklearn` we need `datasets` module.</br>
You could find many datasets in [it's API reference](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets): synthetic, toy and real life datasets (at least they were real life in 2007 😝)</br>
[User guide entry](https://scikit-learn.org/stable/datasets.html) contains main principles of datasets representation and usage.

For now we use [`make_classification` function](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html) to generate normally distributed clusters of different classes.

In [ ]:
from sklearn.datasets import make_classification


data, labels = make_classification(
    n_samples=100,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_classes=3,
    n_clusters_per_class=1,
    random_state=3,
)

data.shape, labels.shape

In [ ]:
from matplotlib.colors import ListedColormap


COLORS = ListedColormap(("red", "blue", "yellow"))


def plot_dataset(data: np.ndarray, labels: np.ndarray, title: str = ""):
    plt.figure(figsize=(7, 7))
    plt.scatter(data[:, 0], data[:, 1], s=100, c=labels, cmap=COLORS)
    plt.title(title)
    plt.grid()
    plt.show()

In [ ]:
plot_dataset(data, labels, "Initial dataset")

To estimate generalizing ability of final algorithm we need to split data.

Note: in real life we need to introduce _validation_ split as well, but for now only train and test is enough.

In [ ]:
from sklearn.model_selection import train_test_split


train_data, test_data, train_labels, test_labels = train_test_split(
    data,
    labels,
    random_state=1,
)

# k Nearest Neighbours algorithm

The heart of this algorithm is ability to find distance to two arbitrary points given.</br>
This could be done directly via calculating distances between every two points in the dataset or with more sophisticated approaches which we will discuss later.

For now let's get familiar with `sklearn` implementation.

## `sklearn.neighbors`

All code related to Nearest Neighbours based algorithms is presented in [`neighbors` submodule](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.neighbors).

**Conprehensive description** of all the approaches is provided in [the corresponding section of User Guide](https://scikit-learn.org/stable/modules/neighbors.html). It worth reading!

For now we only interested in [`KNeighborsClassifier` class](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
KNeighborsClassifier.__mro__

Note presence of the following classes:

* `sklearn.base.ClassifierMixin` - is nessesary to inherit by all the classifiers. Provides metrics and sets `estimator_type`.
* `sklearn.base.BaseEstimator` - common base for all the estimators (regressors, classifiers, transforms, density estimators)

When you implement your custom `sklearn` compatible class, you have to inherit them as well.

Look at `sklearn.base` [module documentation](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.base) to get familiar with all facilities available for each estimator.

In [ ]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(train_data, train_labels)

In [ ]:
from sklearn.metrics import accuracy_score


predictions = knn_clf.predict(test_data)
accuracy_score(test_labels, predictions)

In [ ]:
predictions

### Visualize results

We create a grid in data space (2 dimensional in our case), then predict class for each element of this gird and finally color each pixel in grid with class colour.</br>
In this way we could see all classifier predictions and estimate complexity and sanity of a model.

In [ ]:
def make_meshgrid(
    data: np.ndarray,
    step: float = 0.05,
    border: float = 0.5,
):
    x, y = data[:, 0], data[:, 1]
    x_min, x_max = x.min() - border, x.max() + border
    y_min, y_max = y.min() - border, y.max() + border
    return np.meshgrid(np.arange(x_min, x_max, step), np.arange(y_min, y_max, step))

In [ ]:
LIGHT_COLORS = ListedColormap(("lightcoral", "lightblue", "lightyellow"))


def plot_decision_surface(estimator, train_data, train_labels, test_data, test_labels):
    # fit model
    estimator.fit(train_data, train_labels)

    # create mesh and predict classes for it
    xx, yy = make_meshgrid(train_data)
    flat_predictions = estimator.predict(np.c_[xx.ravel(), yy.ravel()])
    mesh_predictions = flat_predictions.reshape(xx.shape)

    # create subplots
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle(estimator)

    # plot decision surface on the train data
    axes[0].pcolormesh(xx, yy, mesh_predictions, cmap=LIGHT_COLORS, shading="auto")
    axes[0].scatter(train_data[:, 0], train_data[:, 1], s=100, c=train_labels, cmap=COLORS)
    train_acc = accuracy_score(train_labels, estimator.predict(train_data))
    axes[0].set_title(f"Train data, accuracy={train_acc:.2f}")

    # plot decision surface on the test data
    axes[1].pcolormesh(xx, yy, mesh_predictions, cmap=LIGHT_COLORS, shading="auto")
    axes[1].scatter(test_data[:, 0], test_data[:, 1], c=test_labels, s=100, cmap=COLORS)
    test_acc = accuracy_score(test_labels, estimator.predict(test_data))
    axes[1].set_title(f"Test data, accuracy={test_acc:.2f}")

    plt.show()

In [ ]:
for n_neighbors in (1, 2, 3, 5, 10, 20, 30, 40):
    estimator = KNeighborsClassifier(n_neighbors)

    plot_decision_surface(estimator, train_data, train_labels, test_data, test_labels)

Seems good!

## Harder synthetic problem

In [ ]:
data, labels = make_classification(
    n_samples=100,
    n_features=100,
    n_informative=50,
    n_redundant=50,
    n_classes=3,
    n_clusters_per_class=1,
    random_state=42,
)

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(
    data,
    labels,
    test_size=0.3,
    random_state=1,
)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(train_data, train_labels)

In [ ]:
predictions = clf.predict(test_data)
accuracy_score(test_labels, predictions)

not so good...

### Dimesionality issue

In [ ]:
def knn_vs_dimensionality(n_classes: int, dimensions: tuple):
    """Generates synthetic dataset of given dimensions and checks knn algorithm on them."""
    scores = []

    for dim in dimensions:
        data, labels = make_classification(
            n_samples=1000,
            n_features=dim,
            n_informative=dim // 2,
            n_redundant=dim // 2,
            n_classes=n_classes,
            n_clusters_per_class=1,
            random_state=42,
        )

        train_data, test_data, train_labels, test_labels = train_test_split(
            data,
            labels,
            test_size=0.3,
            random_state=1,
        )

        clf = KNeighborsClassifier(n_neighbors=5)
        clf.fit(train_data, train_labels)
        predictions = clf.predict(test_data)

        acc = accuracy_score(test_labels, predictions)
        scores.append(acc)

    plt.figure(figsize=(12, 7))
    plt.plot(dimensions, scores)
    plt.xlabel("Data dimensionality")
    plt.ylabel("Accuracy")
    plt.grid()
    plt.show()

In [ ]:
knn_vs_dimensionality(5, (10, 20, 50, 100, 500, 1000))

# Implementation efficiency

As we said before there are several ways to find nearest neigbours.</br>
`sklearn` implements three of them:

* Brute force
* KD tree
* Ball tree

You could read about underlying data structures in [related section of User Guide](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbor-algorithms).

TODO: consider each implementation training and inference time

# Practice area

Now let's try and train a simple (or not so) kNN classifier on a more complicated dataset.

Widely known [wine dataset](https://archive.ics.uci.edu/ml/datasets/wine) is one of UCI open source datsets.</br>
We will use UCI archive number of times during this course.

In [ ]:
# download dataset
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data > wine.csv

In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv("wine.csv", header=None)
dataset.head()

In [ ]:
X = dataset.drop(0, axis=1).to_numpy()
y = dataset[0].to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)

In [ ]:
# YOUR CODE HERE

# Bonus area

**Those who gets accuracy of at least 0.8 recieves bonus (0.5 max)**

In [ ]:
# YOUR CODE HERE

# Credits <a class="tocSkip">

Authors: [Radoslav Neychev](https://github.com/neychev), [Vladislav Goncharneko](https://github.com/v-goncharenko)